# 第五讲 虚拟内存

## 5.1 虚拟内存的起因

理想中的存储器：更大、更快、更便宜的非易失性存储器。

虚拟内存的目标是充分利用磁盘的资源。

**早期解决方案**

* 如果是程序太大，超过了内存的容量，可以采用手动覆盖（Overlay）的方式解决。
* 如果是程序太多，超过了内存的容量，可以采用自动交换（Swapping）的方式解决。

## 5.2 覆盖技术

**目标**：在较小的可用内存中运行较大的程序，常用于多道程序系统，与分区存储管理配合使用。

**原理**

* 把程序按照其自身逻辑结构，划分为若干个功能上相对独立的程序模块。
* 那些不同时执行的模块共享一块内存区域，按时间先后来运行。
* 必要部分的代码和数据常驻在内存当中。
* 可选部分平时存放在外存中，在需要时才装入内存。

**缺点**：开销问题。

* 程序员需要确定各个模块之间的覆盖关系。
* 覆盖模块从外存装入内存的开销。

## 5.3 交换技术

**目标**：多道程序在内存中时，让正在运行的程序或需要运行的程序获得更多的内存资源。

**方法**：可将暂时不能运行的程序放到外存，从而获得更多的空闲内存空间。

**实现问题**：

* 交换时机的确定：只有内存空间不够或有不够的危险时进行内存换出。
* 交换区的大小：必须足够大以满足所有用户进行的所有内存映像拷贝。
* 程序还如时的重定位：动态地址映射的方法。

**覆盖技术对比交换技术**：目标是一样的，但覆盖技术发生在一个程序内部，而交换技术发生在程序之间。

## 5.4 虚存技术

**为什么要虚存技术？**

* 覆盖技术：增加程序员负担。
* 交换技术：粒度过大，增加了处理器的开销。

**目标**

* 像覆盖技术，把程序的部分内容放在内存中，并由操作系统自动来完成。
* 像交换技术，能够是实现进程在内存与外村之间的交换，并且只对进程的部分内容在内存和外存之间交换。

**程序的局部性原理（Principle of Locality）**：指程序在执行过程中的一个较短时期，所执行的指令地址和指令的操作数地址，分别局限于一定区域，这可以表现为：

* 时间局部性：一条指令的一次执行和下次执行，一个数据的一次访问和下次访问都集中在一个较短时期内。
* 空间局部性：当前指令和邻近的几条指令，当前访问的数据和邻近的几个数据都集中在一个较小的区域内。

**基本概念**：

* 在装入程序时，仅装入必要的页面或段到内存中。
* 在程序执行中，如果需要的指令或数据不在内存中时（称为缺页或缺段），则由处理器通知操作系统将相应的也或段加载入内存，然后继续执行。
* 当内存空间不足时，操作系统会将内存中暂时不用的页或段调出保存在外存中。

**基本特征**：

* 大用户空间：虚拟内存 = 物理内存 + 磁盘。
* 部分交换：与交换技术相比，粒度小，效率更高。
* 不连续的：物理内存分配的不连续，虚拟地址空间使用的不连续。

**虚拟页表项**：

* 驻留位：表示该页是否在内存，1为在内存中。
* 保护位：表示允许对该页做何种操作，读、写、执行。
* 修改位：如果该页被修改过，该位为1，当系统回收该物理页时，根据此位来决定是否把它写回外存。
* 访问位：如果该页被访问过，则设置此位为1，用于页面置换算法。

**缺页中断处理**：

1. 如果内存中有空闲物理页面，则分配物理页帧f，然后转到第4步；否则进入第2步。
2. 采用页面置换算法，选择一个被替换的物理页帧f，原对应逻辑也为q，如果对应页对应的内容被修改修改过，则需写回外存中。
3. 设置q对应的页表项驻留位置0。
4. 将所需访问的页p装入物理页面f当中。
5. 修改p所对应的页表项内容，把驻留位置设为1，把物理页帧设为f。
6. 重新执行被中断的指令。

**后备存储（Backing Store）**

* 一个虚拟地址空间的页面可以被映射到一个文件中的某个位置。
* 代码段：映射到可执行二进制文件。
* 动态加载的共享程序段：映射到动态调用的源文件。
* 其它段：可能被映射到交换文件（Swap File）。

**虚拟内存性能**：有效存储访问时间（Effective Memory Access Time，EAT）。

EAT = 访存时间 $\times$ 页表命中率 + Page Fault 处理时间 $\times$ Page Fault 率。

例子：

* 访问内存时间：10ns
* 磁盘访问时间：5ms
* 参数p = page fault 率
* 参数q = dirty fault 率（物理页被修改，需要写回外存的概率）

$EAT = 10 \times (1 - p) + 5000000 \times p \times (1 + q)$

